<hr style="border:2px solid gray"> </hr>

# MachineLearning
<hr style="border:2px solid gray"> </hr>

# Step 0. Importing packages and Global Settings

In [2]:
# package list
import numpy as np


In [5]:
data = np.load('data.npy')
data.shape

(10542, 64, 64)

# Step 1. Data Loading

# Step 2. 

# Main

Execution Time: 0.08000063896179199
Path: [0, 5, 13, 14, 15, 16, 24, 17, 12, 4, 11, 6, 7, 1, 2, 8, 9, 3, 10, 23, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 18, 19, 20, 21, 22, 42, 43, 60, 59, 58, 57, 56, 55, 50, 51, 49, 48, 47, 46, 54, 53, 45, 44, 52, 73, 67, 63, 74, 76, 77, 80, 81, 86, 87, 91, 93, 98, 101, 100, 99, 104, 105, 106, 112, 107, 108, 114, 118, 115, 119, 116, 121, 128, 127, 126, 125, 124, 123, 120, 117, 113, 122, 129, 111, 97, 92, 88, 68, 64, 61, 65, 69, 70, 66, 62, 75, 78, 82, 83, 84, 85, 79, 71, 72, 89, 90, 94, 95, 96, 103, 102, 110, 109, 130, 0]
Cost: 687.7892183230354
